In [2]:
import torch
from torch import Tensor
from torchvision import transforms
from torch_geometric.nn import GCNConv, global_mean_pool
import torch.nn.functional as F
from torch.utils.data import random_split
from torch_geometric.loader import DataLoader
from torch_geometric.data import Data
from torch_geometric.utils import negative_sampling
from torch.utils.tensorboard import SummaryWriter
import torch.nn as nn
from tqdm import tqdm

import os
from PIL import Image
from torch.utils.data import Dataset

import torch.nn.functional as F
import numpy as np
from sklearn.feature_extraction import image
import cv2
from typing import Tuple, Optional, Union

import kagglehub

from lib.lib import SignatureDataset, image_to_graph
from torch_geometric.nn import GAE

In [3]:
# Hyperparameters
learning_rate = 1e-3
batch_size = 32  # Changed to match your DataLoader batch_size
epochs = 50
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
class SignatureGNN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, embedding_dim):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.conv3 = GCNConv(hidden_channels, embedding_dim)
        
    def forward(self, x: Tensor, edge_index: Tensor, batch: Optional[Tensor] = None) -> Tensor:
        """
        x: Node features [num_nodes, in_channels]
        edge_index: Graph edges [2, num_edges]
        batch: Graph IDs for mini-batch training [num_nodes] - not used for GAE
        """
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index).relu()
        x = self.conv3(x, edge_index)
        
        # Return node-level embeddings for GAE
        # Do NOT use global_mean_pool for GAE - it needs node embeddings
        return x  # [num_nodes, embedding_dim]

In [5]:
def dataset_path():
    path = kagglehub.dataset_download("akashgundu/signature-verification-dataset")
    return os.path.join(path, 'extract')

def transform(**kwargs):
    return transforms.Compose([
        transforms.Grayscale(num_output_channels=kwargs['num_output_channels']),
        transforms.Resize(kwargs['resize']),
        transforms.ToTensor(),
    ])

dataset = SignatureDataset(
    root_dir=dataset_path(),
    transform=transform(num_output_channels=1, resize=(150, 150))
)

Loaded 14626 signature images (genuine + forged)


In [6]:
total_size = len(dataset)
train_size = int(0.8 * total_size)
val_size = total_size - train_size
train_dataset, val_dataset = random_split(
    dataset,
    [train_size, val_size],
    generator=torch.Generator().manual_seed(42)
)
print(f"Dataset sizes - Train: {train_size}, Validation: {val_size}")

Dataset sizes - Train: 11700, Validation: 2926


In [7]:
train_graph = []
val_graph = []

for t, v in tqdm(zip(train_dataset, val_dataset)):
    for train_tensor_image, val_tensor_image in zip(t, v):
        t_graph = image_to_graph(train_tensor_image)
        v_graph = image_to_graph(val_tensor_image)
        train_graph.append(t_graph)
        val_graph.append(v_graph)

2926it [11:48,  4.13it/s]


In [8]:
train_graph = DataLoader(train_graph, batch_size=batch_size, shuffle=True)
val_graph = DataLoader(val_graph, batch_size=batch_size, shuffle=False)

In [12]:
from sklearn.metrics import roc_auc_score, average_precision_score
from torch_geometric.utils import negative_sampling
from datetime import datetime

# Create unique writer for each run
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
writer = SummaryWriter(f'runs/gae_experiment_{timestamp}')

def train_loop(model, graphs, val_graphs, lr, epochs=50, batch=None):
    optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=1e-4)
    
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        
        for graph in tqdm(graphs):
            optimizer.zero_grad()  # Move BEFORE forward pass
            
            # For individual graphs, don't pass batch parameter
            z = model.encode(graph.x, graph.edge_index)
            loss = model.recon_loss(z, graph.edge_index)
            
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        
        # Calculate average training loss
        avg_train_loss = total_loss / len(graphs)
        
        # Validation phase
        model.eval()
        total_val_loss = 0
        total_auc = 0
        total_ap = 0
        
        with torch.no_grad():
            for val_graph in tqdm(val_graphs):
                # For individual graphs, don't pass batch parameter
                z = model.encode(val_graph.x, val_graph.edge_index)
                val_loss = model.recon_loss(z, val_graph.edge_index)
                total_val_loss += val_loss.item()

                # Calculate link prediction accuracy
                pos_edge_index = val_graph.edge_index
                neg_edge_index = negative_sampling(
                    pos_edge_index, 
                    num_nodes=val_graph.x.size(0),
                    num_neg_samples=pos_edge_index.size(1)
                )

                # Get probabilities for positive and negative edges
                pos_pred = model.decoder(z, pos_edge_index, sigmoid=True)
                neg_pred = model.decoder(z, neg_edge_index, sigmoid=True)

                # Create labels and predictions
                y_true = torch.cat([torch.ones(pos_pred.size(0)), torch.zeros(neg_pred.size(0))])
                y_pred = torch.cat([pos_pred, neg_pred])

                # Calculate metrics - THIS IS WHERE AUC AND AP ARE COMPUTED
                auc = roc_auc_score(y_true.cpu(), y_pred.cpu())
                ap = average_precision_score(y_true.cpu(), y_pred.cpu())
                
                total_auc += auc
                total_ap += ap
                
        # Calculate average validation loss
        avg_val_loss = total_val_loss / len(val_graphs)

        # Calculate averages
        avg_auc = total_auc / len(val_graphs)
        avg_ap = total_ap / len(val_graphs)
        
        # Log to TensorBoard (use averages, not last batch)
        writer.add_scalar('Accuracy/AUC', avg_auc, epoch)
        writer.add_scalar('Accuracy/AP', avg_ap, epoch)
        writer.add_scalar('Loss/Train', avg_train_loss, epoch)
        writer.add_scalar('Loss/Val', avg_val_loss, epoch)
        
        print(f'Epoch {epoch+1}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}, AUC: {avg_auc:.4f}, AP: {avg_ap:.4f}')
    
    writer.close()
    return model

In [ ]:
# from torch_geometric.nn import VGAE
input_dim = next(iter(train_graph)).x.shape[1]
hidden_dim = 64
embedding_dim = 128
epochs = 400

encoder = SignatureGNN(input_dim, hidden_dim, embedding_dim)

model = GAE(encoder)
model.load_state_dict(torch.load('gae_malware_model_20250923_220752.pth'))
try:
    gae = train_loop(model, train_graph, val_graph, learning_rate, epochs)
except KeyboardInterrupt:
    print("Saving Model...")
    orch.save(model.state_dict(), "model.pth")
    

100%|██████████████████████████████████████████████████████████████████████████████████| 92/92 [00:42<00:00,  2.15it/s]


Epoch 1, Train Loss: 1.0582, Val Loss: 1.0601, AUC: 0.8702, AP: 0.8362


100%|██████████████████████████████████████████████████████████████████████████████████| 92/92 [00:41<00:00,  2.23it/s]


Epoch 2, Train Loss: 1.0586, Val Loss: 1.0591, AUC: 0.8695, AP: 0.8343


100%|██████████████████████████████████████████████████████████████████████████████████| 92/92 [00:41<00:00,  2.22it/s]


...

100%|██████████████████████████████████████████████████████████████████████████████████| 92/92 [00:43<00:00,  2.11it/s]


Epoch 122, Train Loss: 1.0512, Val Loss: 1.0514, AUC: 0.8762, AP: 0.8410


100%|██████████████████████████████████████████████████████████████████████████████████| 92/92 [00:39<00:00,  2.33it/s]


Epoch 123, Train Loss: 1.0493, Val Loss: 1.0532, AUC: 0.8748, AP: 0.8389


 24%|███████████████████▌                                                              | 22/92 [00:13<00:42,  1.66it/s]

In [11]:
# 2000
torch.save(gae.state_dict(), "model.pth")

NameError: name 'gae' is not defined

In [2]:
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
model_path = f"gae_malware_model_{timestamp}.pth"
torch.save(gae.state_dict(), model_path)
print(f"Model saved to: {model_path}")

NameError: name 'torch' is not defined